In [1]:
#Fresno Data Development Script
#Author: Bill P. Zhai
import geopandas as gpd;
import pandas as pd;
import json;
from shapely.geometry import Point;

#clip fresno city's building models from a list of open city model files
def clipOpenCityModels(fileList, cityBoundsFile):
    cityBoundsGdf = gpd.read_file(cityBoundFolder+"/"+cityBoundsFile+".geojson");
    cityBoundsGdf.crs = {'init': 'epsg:2227'};
    #print(cityBoundsGdf);
    openCityGdfList = list();
    for file in fileList:
        openCityGdf = gpd.read_file(openCityGmlFolder+"/"+file+".geojson");
        openCityGdf.crs = {'init': 'epsg:4326'};
        openCityGdf = openCityGdf.to_crs(epsg=2227);
        #spatial join:
        openCityGdfWithinFresno = gpd.sjoin(openCityGdf, cityBoundsGdf,  how="inner", op='intersects');
        openCityGdfWithinFresno = openCityGdfWithinFresno.drop(['index_right', 'CITY_COUNT', 'SHAPE_AREA', 'SHAPE_LEN'], axis=1);#drop index_right column cuz it's both unnecessary and confusing 
        #print(openCityGdfWithinFresno);
        openCityGdfList.append(openCityGdfWithinFresno);
        print(file+" done!");
    resultGdf = gpd.GeoDataFrame( pd.concat(openCityGdfList, ignore_index=True));
    #write Gdf to file
    resultGdf.to_file("FresnoCityModels.geojson", driver='GeoJSON');
    return resultGdf;

#clip UseType geojson to Fresno City Boundary
def clipUseType(useTypeFile, cityBoundsFile):
    useTypeGdf = gpd.read_file(parcelLandUseFolder+"/"+useTypeFile+".geojson");
    useTypeGdf.crs = {'init': 'epsg:2227'};
    boundsGdf = gpd.read_file(cityBoundFolder+"/"+cityBoundsFile+".geojson");
    boundsGdf.crs = {'init': 'epsg:2227'};
    #spatial join:
    print("start join");
    resultGdf = gpd.sjoin(useTypeGdf, boundsGdf, how="inner", op='intersects');
    resultGdf.to_file("FresnoCityParcelUseTypes.geojson", driver='GeoJSON');
    return resultGdf;

def mergeUseTypeToCityModel(cityModelFile, cityParcelUseTypeFile):
    cityModelGdf = gpd.read_file(cityModelFile+".geojson");
    cityModelGdf.crs = {'init': 'epsg:2227'};
    useTypeGdf = gpd.read_file(cityParcelUseTypeFile+".geojson");
    useTypeGdf.crs = {'init': 'epsg:2227'};
    lngLat = [Point(xy) for xy in zip(cityModelGdf['longitude'], cityModelGdf['latitude'])];
    cityModelGdf['LngLat'] = lngLat;
    cityModelGdf = cityModelGdf.set_geometry('LngLat');
    cityModelGdf.crs = {'init': 'epsg:2227'};
    #spatial join:
    resultGdf = gpd.sjoin(cityModelGdf, useTypeGdf, how="left", op='intersects');
    print(resultGdf);
    resultGdf.crs = {'init': 'epsg:2227'};
    resultGdf.to_file("CityModelWithUseType.geojson", driver='GeoJSON');
    return resultGdf;

In [2]:
#global variables:
cityBoundFolder = "Bounds"; #contains Fresno bounds .geojson file
openCityGmlFolder = "OpenCityGml"; #contains open city model .json file
parcelLandUseFolder = "Fresno_parcel_landuse_2016"; #contains landuse by APN .geojson file
parcelAPNFolder = "Parcels"; #contains parcel polygon and APN .geojson file

In [3]:
clipOpenCityModels(['FresnoCountyCityGml_000','FresnoCountyCityGml_001','FresnoCountyCityGml_002','FresnoCountyCityGml_003','FresnoCountyCityGml_004','FresnoCountyCityGml_005','FresnoCountyCityGml_006','FresnoCountyCityGml_007','FresnoCountyCityGml_008'], 'FresnoBounds')

/Users/pengyuanzhai/anaconda3/lib/python3.7/site-packages/geopandas/tools/sjoin.py:44: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')


FresnoCountyCityGml_000 done!
FresnoCountyCityGml_001 done!
FresnoCountyCityGml_002 done!
FresnoCountyCityGml_003 done!
FresnoCountyCityGml_004 done!
FresnoCountyCityGml_005 done!
FresnoCountyCityGml_006 done!
FresnoCountyCityGml_007 done!
FresnoCountyCityGml_008 done!


,gml_id,hash,ubid,state,county,grid,latitude,longitude,area,height,height_source,fp_source,measuredHeight,measuredHeight_uom,geometry
0,ODU4MlI1MjMrOi0yMDYzMTMxMTk1,ODU4MlI1MjMrOi0yMDYzMTMxMTk1,8582R523+2X-0-1-1-0,California,6019,8582R523+,36.800022,-119.845005,258.097929,11.37,msfp-2017,msfp-2017,11.37,urn:ogc:def:uom:UCUM::m,(POLYGON Z ((6753433.265209929 1750352.2984065...
1,ODU4MlI1MjMrOjE1MTQ3MTQxMDk,ODU4MlI1MjMrOjE1MTQ3MTQxMDk,8582R523+83-0-1-1-1,California,6019,8582R523+,36.800755,-119.847339,281.162610,10.61,msfp-2017,msfp-2017,10.61,urn:ogc:def:uom:UCUM::m,(POLYGON Z ((6752742.291550246 1750620.4476972...
2,ODU4MlI1MjMrOjUzMTQ4MjU0NA,ODU4MlI1MjMrOjUzMTQ4MjU0NA,8582R523+5X-1-1-1-0,California,6019,8582R523+,36.800463,-119.845025,299.653281,12.83,msfp-2017,msfp-2017,12.83,urn:ogc:def:uom:UCUM::m,(POLYGON Z ((6753429.579570424 1750517.0196719...
3,ODU4MlI1MjMrOjMzMDA0OTU4Nw,ODU4MlI1MjMrOjMzMDA0OTU4Nw,8582R523+F5-1-0-1-1,California,6019,8582R523+,36.801189,-119.847121,313.382521,11.46,msfp-2017,msfp-2017,11.46,urn:ogc:def:uom:UCUM::m,(POLYGON Z ((6752801.244463062 1750776.0229575...
4,ODU4MlI1MjMrOi02NjY1MTQ0NTc,ODU4MlI1MjMrOi02NjY1MTQ0NTc,8582R523+FG-1-1-1-1,California,6019,8582R523+,36.801172,-119.846154,305.680650,12.08,msfp-2017,msfp-2017,12.08,urn:ogc:def:uom:UCUM::m,(POLYGON Z ((6753145.878237226 1750761.4282722...
5,ODU4MlI1MjMrOjE3Mjc3ODI2Njc,ODU4MlI1MjMrOjE3Mjc3ODI2Njc,8582R523+FC-0-1-1-1,California,6019,8582R523+,36.801167,-119.846397,321.137097,12.08,msfp-2017,msfp-2017,12.08,urn:ogc:def:uom:UCUM::m,(POLYGON Z ((6753073.686010569 1750765.9818895...
6,ODU4MlI1MjMrOjE3NjMwOTA5Nzg,ODU4MlI1MjMrOjE3NjMwOTA5Nzg,8582R523+5C-1-0-1-1,California,6019,8582R523+,36.800452,-119.846473,278.425047,11.76,msfp-2017,msfp-2017,11.76,urn:ogc:def:uom:UCUM::m,(POLYGON Z ((6753047.268458018 1750517.2587615...
7,ODU4MlI1MjMrOjE3MjI4OTU0NzQ,ODU4MlI1MjMrOjE3MjI4OTU0NzQ,8582R523+2F-0-1-1-1,California,6019,8582R523+,36.800014,-119.846267,293.476134,12.05,msfp-2017,msfp-2017,12.05,urn:ogc:def:uom:UCUM::m,(POLYGON Z ((6753106.357998836 1750359.5011997...
8,ODU4MlI1MjMrOi0yMTk5MjM5NDI,ODU4MlI1MjMrOi0yMTk5MjM5NDI,8582R523+F7-1-0-1-1,California,6019,8582R523+,36.801170,-119.846869,274.205491,11.50,msfp-2017,msfp-2017,11.50,urn:ogc:def:uom:UCUM::m,(POLYGON Z ((6752928.521954995 1750778.3904588...
9,ODU4MlI1MjMrOjk3MjQwMjg2Ng,ODU4MlI1MjMrOjk3MjQwMjg2Ng,8582R523+2M-0-0-1-1,California,6019,8582R523+,36.800017,-119.845848,292.729592,11.90,msfp-2017,msfp-2017,11.90,urn:ogc:def:uom:UCUM::m,(POLYGON Z ((6753230.46299394 1750360.43696273...


In [3]:
usetypeClp = clipUseType("UseTypeZoom", "FresnoBounds");
print("done");

start join
done


In [31]:
mergeUseTypeToCityModel('FresnoCityModels', 'ParcelLandUse')

IndexError: list index out of range

,measuredHeight,area,longitude,latitude,geometry,index_right,APN,APN_D,HSENO,STR,COUNTYUSE,USETYPE,ZONING
46,11.17,256.081290,-119.731215,36.769336,POINT (6786867.486447326 1739412.223013624),30779,49424X92,494-24X-92,None,None,None,None,None
46,11.17,256.081290,-119.731215,36.769336,POINT (6786867.486447326 1739412.223013624),30781,49424X92,494-24X-92,None,None,None,None,None
621,13.02,301.272792,-119.729963,36.769925,POINT (6787232.462944718 1739629.693057405),30781,49424X92,494-24X-92,None,None,None,None,None
1559,13.80,337.859253,-119.731023,36.769959,POINT (6786921.863988086 1739639.519319319),30781,49424X92,494-24X-92,None,None,None,None,None
2759,13.16,412.510950,-119.730003,36.769221,POINT (6787222.85511136 1739373.270355404),30781,49424X92,494-24X-92,None,None,None,None,None
2912,12.78,313.975825,-119.729918,36.769429,POINT (6787247.13051103 1739449.208001751),30781,49424X92,494-24X-92,None,None,None,None,None
3352,12.09,402.296653,-119.731244,36.769750,POINT (6786857.753382517 1739562.89058442),30781,49424X92,494-24X-92,None,None,None,None,None
4100,13.46,437.294111,-119.730411,36.769701,POINT (6787101.904631943 1739547.055201588),30781,49424X92,494-24X-92,None,None,None,None,None
4834,13.68,401.526307,-119.729862,36.768983,POINT (6787264.870617026 1739286.954452755),30781,49424X92,494-24X-92,None,None,None,None,None
5661,13.27,413.683877,-119.729691,36.769742,POINT (6787312.686155513 1739563.718499817),30781,49424X92,494-24X-92,None,None,None,None,None


In [11]:
json.load("OpenCityGml/California-06013-000.json")

AttributeError: 'str' object has no attribute 'read'

In [8]:
gpd.read_file(cityBoundFolder+"/"+"FresnoBounds"+".geojson")

,CITY_COUNT,SHAPE_AREA,SHAPE_LEN,geometry
0,city,3.240798e+09,1.156110e+06,"(POLYGON ((6742163.16681591 1715201.058881051,..."


DriverError: 'OpenCityGml/California-06013-001.json' not recognized as a supported file format.